<a href="https://colab.research.google.com/github/MuhammadIrzam447/MultiModel/blob/master/Valid_30(a).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !gdown https://drive.google.com/uc?id=1p1qz1lDDwqrW-lTa11liOBMuWwIT8IP4 # Joint

In [ ]:
# !gdown https://drive.google.com/uc?id=1mt8xgtCsp1866_JtVUXSBKV5N-vjwV7Z # Fused Dataset

In [ ]:
# !gdown https://drive.google.com/uc?id=16Mx_3KupmbN93utF1ICsDibmrTDcOE3P # Model

In [ ]:
# !unrar x "/content/multi_model-images-val.rar"

In [ ]:
# !unzip "/content/mulitmodal_img_enc_txt.zip"

# Load Model


In [1]:
!pip install transformers evaluate datasets
import requests
import torch
from PIL import Image
import torchvision
import torch.nn as nn
from torchvision import datasets, transforms
import torchvision.models as models
from tqdm import tqdm

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [3]:
import os
save_dir = '/content/'
load_path = os.path.join(save_dir, 'fused_model.pth')

# Create an instance of the ResNet model
resnet = torchvision.models.resnet101(pretrained=False)
resnet.fc = nn.Linear(2048, 52) # Choose the number of output classses as per your model

# Load the saved model parameters
resnet.load_state_dict(torch.load(load_path))
# resnet.load_state_dict(torch.load(load_path, map_location=torch.device('cpu')))

resnet.eval()
resnet.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

# Partial Missing Modality Experiment

In [4]:
from datasets import load_dataset
from datasets import concatenate_datasets

In [5]:
# Load the Fused Dataset
ds_f = load_dataset("imagefolder", data_dir="/content/images-val", split="train")

Resolving data files:   0%|          | 0/21869 [00:00<?, ?it/s]

In [6]:
ds_f

Dataset({
    features: ['image', 'label'],
    num_rows: 21869
})

In [7]:
# Desired Percentage = Total - Missing
import math
desired_percentage = 0.5

In [8]:
from collections import Counter

class_counts = Counter(ds_f['label'])
print(class_counts)

Counter({51: 2200, 21: 1926, 1: 1879, 14: 1555, 43: 1352, 33: 1196, 6: 1069, 13: 842, 27: 697, 28: 695, 8: 663, 9: 642, 23: 634, 41: 522, 26: 450, 48: 379, 2: 345, 31: 331, 39: 327, 22: 281, 24: 270, 44: 254, 36: 238, 17: 206, 40: 203, 25: 196, 5: 176, 49: 159, 42: 157, 7: 148, 35: 148, 29: 146, 37: 146, 38: 142, 10: 136, 4: 132, 47: 116, 12: 112, 50: 106, 30: 85, 46: 80, 34: 73, 0: 59, 18: 59, 32: 54, 16: 51, 3: 48, 19: 43, 45: 37, 15: 36, 11: 35, 20: 33})


In [9]:
selected_indices = {label: [] for label in class_counts.keys()}
print(selected_indices)

{0: [], 1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: [], 10: [], 11: [], 12: [], 13: [], 14: [], 15: [], 16: [], 17: [], 18: [], 19: [], 20: [], 21: [], 22: [], 23: [], 24: [], 25: [], 26: [], 27: [], 28: [], 29: [], 30: [], 31: [], 32: [], 33: [], 34: [], 35: [], 36: [], 37: [], 38: [], 39: [], 40: [], 41: [], 42: [], 43: [], 44: [], 45: [], 46: [], 47: [], 48: [], 49: [], 50: [], 51: []}


In [10]:
# Iterate through the dataset to select indices for each class
for i, label in enumerate(ds_f["label"]):
    if len(selected_indices[label]) < math.ceil(class_counts[label] * desired_percentage):
        selected_indices[label].append(i)

In [11]:
# selected_indices

In [12]:
# Flatten the selected indices list
selected_indices = [idx for indices in selected_indices.values() for idx in indices]

In [13]:
# selected_indices

In [14]:
# Filter the dataset to select the desired samples
fused_ds = ds_f.select(selected_indices)

In [15]:
fused_ds

Dataset({
    features: ['image', 'label'],
    num_rows: 10946
})

In [16]:
# Load the Joint Dataset and Filter out Just the Images
ds_joint = load_dataset("imagefolder", data_dir="/content/mulitmodal_img_enc_txt/images-val", split="train")

Resolving data files:   0%|          | 0/43738 [00:00<?, ?it/s]

In [17]:
# Use _4.png for Filtering out Just Images

import os
def filter_funtion(example):
    img = example["image"]
    filename = os.path.basename(img.filename)

    return filename.endswith("_4.png")

In [18]:
ds_imgs = ds_joint.filter(filter_funtion)

In [19]:
ds_imgs

Dataset({
    features: ['image', 'label'],
    num_rows: 21869
})

In [20]:
# for i in range(10,100):
#     sample = ds_imgs[i]
#     img = sample["image"]
#     filename = os.path.basename(img.filename)
#     print(f"File Name: {filename}")

In [21]:
all_indices = range(len(ds_imgs))
print(all_indices)

range(0, 21869)


In [22]:
remaining_indices = [idx for idx in all_indices if idx not in selected_indices]

In [23]:
# remaining_indices

In [24]:
missing_ds = ds_imgs.select(remaining_indices)

In [25]:
missing_ds

Dataset({
    features: ['image', 'label'],
    num_rows: 10923
})

In [26]:
# Combine Both Datasets

In [27]:
ds = concatenate_datasets([fused_ds, missing_ds])

In [28]:
ds

Dataset({
    features: ['image', 'label'],
    num_rows: 21869
})

# Pre-Processing and Validation Loop

In [29]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image

In [30]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  std=[0.229, 0.224, 0.225])])

In [31]:
class CustomDataset(Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        image = self.dataset[idx]["image"]
        label = self.dataset[idx]["label"]

        if self.transform:
            image = self.transform(image)

        return {"image": image, "label": label}

In [32]:
dataset = CustomDataset(ds, transform=transform)

In [33]:
batch_size = 32
test_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

In [34]:
true_labels = []
predicted_labels = []

resnet.eval()

with torch.no_grad():
    for batch in test_loader:
        inputs = batch["image"]
        labels = batch["label"]

        inputs = inputs.to(device)

        outputs = resnet(inputs)

        _, predicted = torch.max(outputs, 1)

        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(predicted.cpu().numpy())

In [35]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report

accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels, average='weighted')
recall = recall_score(true_labels, predicted_labels, average='weighted')
f1 = f1_score(true_labels, predicted_labels, average='weighted')
# AUROC_score = roc_auc_score(true_labels, predicted_labels)

In [36]:
print("Accuracy: ", accuracy)
print("precision: ", precision)
print("recall", recall)
print("f1_score: ", f1)
# print("AUROC_score: ", AUROC_score)

Accuracy:  0.9357080799304952
precision:  0.9360630166262038
recall 0.9357080799304952
f1_score:  0.9352769725613237


In [37]:
print(classification_report(true_labels, predicted_labels))

              precision    recall  f1-score   support

           0       0.51      0.56      0.53        59
           1       0.98      0.95      0.96      1879
           2       0.97      0.89      0.93       345
           3       0.98      0.88      0.92        48
           4       0.95      0.99      0.97       132
           5       0.91      0.83      0.87       176
           6       0.95      0.98      0.96      1069
           7       0.95      0.95      0.95       148
           8       0.97      0.97      0.97       663
           9       0.95      1.00      0.97       642
          10       0.87      0.95      0.91       136
          11       0.86      0.91      0.89        35
          12       0.82      0.94      0.88       112
          13       0.94      0.95      0.95       842
          14       0.96      0.99      0.97      1555
          15       0.72      0.72      0.72        36
          16       0.91      0.94      0.92        51
          17       0.87    

In [38]:
cm = confusion_matrix(true_labels, predicted_labels)
print(cm)

[[  33    1    0 ...    1    0    2]
 [   4 1780    0 ...    0    0   14]
 [   0    0  306 ...    0    0    0]
 ...
 [   1    1    0 ...   99    0    0]
 [   0    0    0 ...    0  103    0]
 [   0    1    0 ...    0    0 2170]]
